In [2]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import date, datetime
from sklearn import preprocessing, model_selection
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

Using TensorFlow backend.


In [3]:
#faz conexão com o google drive, onde estão os dados
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Carrega o diretório do dataset
file_path = "/content/drive/My Drive/Hackaturin/hackaturing.dsv"


In [5]:
#Lê o dataset 
df = pd.read_csv(file_path, sep='|')


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#Filtra informações sobre paciente que já realizou gastroplastia e também de pacientes que foram diagnosticados com obesidade
listacod = [31001076,31002218,31002390,84430095,84430109,84430460,84430540,84430478] #IDs referentes a Gastroplastia
cirurgia = df[df['base_hackaturing.servico'].isin(listacod)]

listacod = ['E660','E661','E662','E668','E669'] #IDs referentes a Obesidade
obesidade = df[df['base_hackaturing.cid'].isin(listacod)]

In [0]:
#Limpa informações de usuarios repetidas
obesidades = obesidade.drop_duplicates(subset='base_hackaturing.id_beneficiario', keep='first')

cirurgia = cirurgia.drop_duplicates(subset='base_hackaturing.id_beneficiario', keep='first')

In [0]:
#Relaciona pacientes que foram diagnosticados com obesidade e que já realizaram gastroplastia
count = 0
lista = np.zeros(len(obesidade))
#merged = pd.merge(obesidades, cirurgia, how='inner', on='base_hackaturing.id_beneficiario', validate='one_to_one')
#lista = merged['base_hackaturing.id_beneficiario']
for j in obesidade['base_hackaturing.id_beneficiario']:
  for i in cirurgia['base_hackaturing.id_beneficiario']:
    if i == j:
      lista[count] = 1
  count += 1

#As labels são um quando o paciente realizou a cirurgia e zero se não realizou
Y = lista

In [0]:
#Filtra informações necessárias para a modelagem
sexo = obesidade['base_hackaturing.sexo'].tolist()
nascimento = obesidade['base_hackaturing.data_nascimento'].fillna(0).tolist()
uf = obesidade['base_hackaturing.uf'].tolist()
carater = obesidade['base_hackaturing.carater_atendimento'].tolist()
obesity = np.ones(len(obesidade))

In [0]:
#Calcula a idade do paciente
today = datetime.today()
age = np.zeros(len(nascimento))
for i in range(len(nascimento)):
  if nascimento[i] == 0:
    continue
  variavel = datetime.strptime(str(nascimento[i]), '%Y-%m-%d')
  idade = today.year - variavel.year - ((today.month, variavel.day) < (variavel.month, variavel.day))
  nascimento[i] = idade
  


In [0]:
#Prepara os dados para o modelo e cria a matriz de variáveis
nascimento = np.array(nascimento)

le = preprocessing.LabelEncoder()
le.fit(carater)
carater = le.transform(carater) 

le = preprocessing.LabelEncoder()
le.fit(sexo)
sexo = le.transform(sexo) 

le = preprocessing.LabelEncoder()
le.fit(uf)
uf = le.transform(uf) 

X = np.matrix([nascimento, sexo, uf, obesity, carater]).T

In [0]:
#Normaliza o dado 
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [0]:
#Divide o dataset entre conjunto de treino e validação
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.15, random_state=42)


In [48]:
#create the  KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=4)

#train model
clf.fit(X_train, Y_train)
float_formatter = lambda x: "%.2f" % x
#print(float_formatter(clf.score(X_validation, Y_validation)*100))
#predict
print("Acuracia media no conjunto de treinamento: "+ float_formatter(clf.score(X_train, Y_train)*100)+"%")

print("Acuracia media no conjunto de validacao: "+ float_formatter(clf.score(X_validation, Y_validation)*100)+"%")


Acuracia media no conjunto de treinamento: 80.89%
Acuracia media no conjunto de validacao: 80.09%
